<a href="https://colab.research.google.com/github/Maruddo/AI-Notebooks/blob/main/skin_cancer_model/sc_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Skin Cancer Model

## Import libraries

In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np
import torch
from torch import nn
from torchvision import datasets, transforms
from torch.utils.data import TensorDataset, Dataset, DataLoader, random_split
from torchvision.transforms import ToTensor, Lambda
import torchvision.models as models

# Import & Transform dataset

In [ ]:
def import_dataset(dataset):
    transform = transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor()
        ])

    train_data = datasets.ImageFolder(root=f"{dataset}/train",transform=transform)
    test_data = datasets.ImageFolder(root=f"{dataset}/test",transform=transform)

    return train_data, test_data

# Prepare DataLoader
## Create DataLoaders

In [ ]:
def create_dataloader(training_data, test_data, batch_size):
    train_dataloader = DataLoader(training_data, batch_size, shuffle=True)
    test_dataloader = DataLoader(test_data, batch_size, shuffle=False)

    return train_dataloader, test_dataloader

# Select Device

In [ ]:
def select_device():
    device = (
        "cuda"
        if torch.cuda.is_available()
        else "mps"
        if torch.backends.mps.is_available()
        else "cpu"
    )
    return device

# Define the model

In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(in_channels=3,out_channels=32,kernel_size=5,stride=2),
            nn.MaxPool2d(kernel_size=2,stride=2,padding=0),
            nn.Conv2d(in_channels=32,out_channels=64,kernel_size=3,stride=2),
            nn.MaxPool2d(kernel_size=2,stride=2,padding=0),
                nn.Flatten(),
                nn.Linear(10816,5000),
                nn.LeakyReLU(),
                nn.Linear(5000,2000),
                nn.LeakyReLU(),
                nn.Linear(2000,200),
                nn.LeakyReLU(),
                nn.Linear(200,2)
        )

    def forward(self, X):
        logits = self.features(X)
        return logits

def load_model(weights, device):
    model = NeuralNetwork().to(device)
    if os.path.exists(weights):
         try:
             model.load_state_dict(torch.load(weights, mmap=True, weights_only=True), assign=True)
         except:
             print("None weights were loaded")
    return model

# Train & Test

In [ ]:
def train_loop(dataloader, model, loss_fn, optimizer, device):
    size = len(dataloader.dataset)
    # Set the model to training mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()


        if batch % 25 == 0:
            loss, current = loss.item(), batch * batch_size + len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test_loop(dataloader, model, loss_fn, device, best_accuracy, best_weights):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    accuracy = correct / size

    print(f"Test Error: \n Accuracy: {(100*accuracy):>0.1f}%, Avg loss: {test_loss:>8f} \n")

    if should_save_best_model(accuracy, best_accuracy):
        best_accuracy = accuracy
        best_weights = model.state_dict()

    return best_accuracy, best_weights

## Save only the best weights (optional and not useful for all cases)

In [ ]:
def should_save_best_model(accuracy, best_accuracy):
    return accuracy > best_accuracy

# Main function

In [ ]:
def __main__(dataset,learning_rate,batch_size,epochs):
    loss_fn = nn.CrossEntropyLoss()
    device = select_device()
    model = load_model('model_weights.pth', device)
    best_accuracy = 0.0
    best_weights = None

    train_on = input("Do you want to train? Yes(y), No(enter)")
    if train_on:
        training_data, test_data = import_dataset(dataset)
        train_dataloader, test_dataloader = create_dataloader(training_data,test_data,batch_size)
        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
        next_train = True
        while next_train:
            for t in range(epochs):
                print(f"Epoch {t+1}\n-------------------------------")
                train_loop(train_dataloader, model, loss_fn, optimizer, device)
                best_accuracy, best_weights = test_loop(test_dataloader, model, loss_fn, device, best_accuracy, best_weights)

            print(f"Best accuracy: {(100 * best_accuracy):.1f}% with learning rate {learning_rate}")
            save_on = input("Do you want to save? Yes(Any), No(enter): ")
            if save_on:
              print("Done! Saving process...")
              torch.save(best_weights, 'model_weights.pth')

            stop_train = input("Should we continue training? No(Any) Yes(Enter) -> ")
            if stop_train:
                next_train = False
            else:
                change_lr = input("Change the learning rate? Yes(Any) No(Enter) -> ")
                if change_lr:
                    try:
                        learning_rate=float(input("New learning rate: "))
                    except:
                        print("Ignoring new learning rate")
                change_epochs = input(f"Change the number of epochs ({epochs})? Yes(Any) No(Enter) -> ")
                if change_epochs:
                    try:
                        epochs=int(input("New number of epochs: "))
                    except:
                        print("Ignoring new learning rate")
    else:
        test_loop(test_dataloader, model, loss_fn, device)

# Call main and set globals

In [ ]:
if __name__ == '__main__':
    dataset = 'melanoma_ds'
    learning_rate = 0.0008
    batch_size = 500
    epochs = 10
    __main__(dataset,learning_rate,batch_size,epochs)

Do you want to train? Yes(y), No(enter) y


Epoch 1
-------------------------------
loss: 0.189540  [  500/ 9605]
Test Error: 
 Accuracy: 89.2%, Avg loss: 0.275762 

Epoch 2
-------------------------------
loss: 0.253691  [  500/ 9605]
Test Error: 
 Accuracy: 88.4%, Avg loss: 0.257420 

Epoch 3
-------------------------------
loss: 0.260218  [  500/ 9605]
Test Error: 
 Accuracy: 88.6%, Avg loss: 0.243113 

Epoch 4
-------------------------------
loss: 0.205118  [  500/ 9605]
Test Error: 
 Accuracy: 88.1%, Avg loss: 0.252951 

Epoch 5
-------------------------------
loss: 0.237253  [  500/ 9605]
Test Error: 
 Accuracy: 89.6%, Avg loss: 0.241601 

Epoch 6
-------------------------------
loss: 0.212053  [  500/ 9605]
Test Error: 
 Accuracy: 89.6%, Avg loss: 0.244763 

Epoch 7
-------------------------------
loss: 0.229738  [  500/ 9605]
Test Error: 
 Accuracy: 89.8%, Avg loss: 0.238783 

Epoch 8
-------------------------------
loss: 0.172356  [  500/ 9605]
Test Error: 
 Accuracy: 90.1%, Avg loss: 0.242024 

Epoch 9
----------------

Should we continue training? No(Any) Yes(Enter) ->  n
